## Section 1 — Report Header & Hypothesis

- API Data Report
- Alaina Norris 
- Date: 2025-10-08

### Hypothesis

Accounts that have more followers also tend to have more posts than accounts with less followers.

### Theoretical Rationale

The reasoning behind my hypothesis of "Accounts that have more followers also tend to have more posts than accounts with less followers." is that people tend to follow people who post more, since then it is easier to keep up with their posts. More popularly followed people such as brands and celebrities also tend to have more followers, and with that also have more things that they post. I think that people who have more followers would have more posts since people are less likely to follow someone who does not post, since then there is not much of a reason to follow the account in order to keep up with them.

### Statistical Application

In order to test this hypothesis statistically, I would like to pull the data of accounts number of followers, as well as the acccounts number of posts. Then I should be able to see (most likely though a graph) that the accounts who tend to have a higher number of posts also have a higher number of followers. If this were to be turned into a graph it would be a positive correlation between followers and posts, meaning the people who post much less also have less followers. The important columns for this would be Account, # of posts, # of followers. 

## Section 2 — Endpoint Plan (Design Your Data Collection)

The Bluesky API endpoints I am identfiying and planning to use include:

- app.bsky.actor.searchActors 
- app.bsky.actor.getProfiles

These endpoints (while I may not use all of them) will allow me to see the information and get the accounts I need as well as get my data in order to analyze it. I will also be able to get the profile information for the different accounts I am using to analyze. This should include the number of followers that the account has, and the number of posts that the account has also. 

In order to find whether accounts that usually post more have more followers, I will need to use the key request parameters of actors / postsCount / followersCount/ and most likley handle. These will be able to show me the connection between the number of posts and the number of followers that the users have, and may be helpful for me to make some form of chart out of the data for a visual. 


### Reliabilit and Bias

Discuss how the data might be reliable and unreliable. Consider:

Some of the data may be unreliable, especially if the accounts have certain privacy settings or they have hidden data. If an account is too new then it may not have an accurate number for the followers and posts. There may be accounts that are outliers and do not follow my expected pattern for this hypothesis such as bot accounts or spam accounts, who may not have any or many followers but post many posts. If there are deleted posts then its possible that the accounts gained lots of followers from their posting and then once they deleted the posts they retained the followers. These datapoints are fairly ethical to collect, as the number of posts and followers that an account has are typically things you can see even on private accounts. 


### Limitations

Some caveats in the response objects for this could be if there were delayed counts, since it is possible that there is not a constantly updated information and it is not exactly accurate. There could also be some missing data. There is a chance of some missing data or some caveats with the follower and posts number information especially if there were something like a new account / bot account/ or account that is under inspection for violations of standards. 

## Section 3 Data Collection

Collect posts that match a query. Adjust QUERY, MAX_POSTS, and any filters your hypothesis requires.

### Data Collection (Endpoint 1):

In [56]:
## used code and notes from previous class on sept 29 for help in this section.
import requests
import time
import json as js
import pandas as pd

BASE_URL = "https://api.bsky.app/xrpc"

In [87]:
## used code and notes from previous class on sept 29 for help in this section.
BASE_URL = "https://bsky.social/xrpc"
endpoint = f"{BASE_URL}/app.bsky.feed.searchPosts"

params = {
    "q": "social media",   
    "limit": 10            
}

resp = requests.get(endpoint, params=params, headers=headers)

print("Status:", resp.status_code)

data = resp.json()
print("Top level keys:", list(data.keys()))

Status: 401
Top level keys: ['error', 'message']


In [88]:
## used code and notes from previous class on sept 29 for help in this section.
posts = data.get("posts", []) or data.get("feed", [])
collected = []
for post in posts:
    author = post.get("author", {})

cursor = data.get("cursor")

In [89]:
rows = []
for p in posts:
    author = p.get("author", {})
    record = p.get("record", {})

    stats = {
        "post_uri": p.get("uri"),
        "likeCount": p.get("likeCount"),
        "repostCount": p.get("repostCount"),
        "author_handle": author.get("handle"),
        "author_displayName": author.get("displayName"),
    }
    rows.append(stats)

In [90]:
## used code and notes from previous class on sept 29 for help in this section.

#sept 29th code below for help.
### Get author profiles for these dids
#all_profiles = []
#for d in unique_dids:
 #   params = []
 #   params.append(("actor", d))
  #  r = requests.get(f"{BASE_URL}/app.bsky.actor.getProfile", params=params, timeout = 30)
  #  data = r.json()
   # print(js.dumps(data, indent = 2))

    # append this profile to our list all_profiles
   # all_profiles.append({
    #    "did": data.get("did"), 
#        "handle": data.get("handle"), 
    #    "displayName" : data.get("displayName"),
     #   "folllowersCount": data.get("followersCount"),
     #   "followsCount": data.get("followsCount"),
     #   "postCount": data.get("postCount"),
  #  })

all_profiles_df = pd.DataFrame(all_profiles)
all_profiles_df.head(5)


all_profiles = []

for d in unique_dids:
    try:
        params = {"actor": d}
        r = requests.get(f"{BASE_URL}/app.bsky.actor.getProfile", params=params, timeout=30)
        r.raise_for_status()
        data = r.json()

        # append this profile to our list all_profiles
        all_profiles.append({
            "did": data.get("did"),
            "handle": data.get("handle"),
            "displayName": data.get("displayName"),
            "followersCount": data.get("followersCount"), 
            "followsCount": data.get("followsCount"),
            "postsCount": data.get("postsCount"),
        })

    # I used Chat GPT for this part below, as when I did not have this section I was getting an error saying it "expected an exception"
    # it says without this line of code there would be an error and it would stop running when it got to the first profile with an issue
    except Exception as e:
        print(f"Error fetching profile for {d}: {e}")

# Convert to DataFrame
all_profiles_df = pd.DataFrame(all_profiles)

# this section of code results in all errors?????

Error fetching profile for did:plc:wmkln3rotsdsh2jxwsqry4ge: 401 Client Error: Unauthorized for url: https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=did%3Aplc%3Awmkln3rotsdsh2jxwsqry4ge
Error fetching profile for did:plc:3drlkjem5kndj5jskrbprrjd: 401 Client Error: Unauthorized for url: https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=did%3Aplc%3A3drlkjem5kndj5jskrbprrjd
Error fetching profile for did:plc:d7jn5n4nawuc7ijk2ew7aukj: 401 Client Error: Unauthorized for url: https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=did%3Aplc%3Ad7jn5n4nawuc7ijk2ew7aukj
Error fetching profile for did:plc:rjscsr7qexxigyhnjctym4bk: 401 Client Error: Unauthorized for url: https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=did%3Aplc%3Arjscsr7qexxigyhnjctym4bk
Error fetching profile for did:plc:mvw7btcxe6jgznss25lcmus7: 401 Client Error: Unauthorized for url: https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=did%3Aplc%3Amvw7btcxe6jgznss25lcmus7
Error fetching profile fo

e.g. app.bsky.feed.searchPosts Flatten key fields from Bluesky PostView objects.

### Data Collection (Endpoint 2):

I believe this was done in the above section??? 

## Section 4 — Build DataFrames

In [94]:
# used ## Merge posts and profile dataframes from septmber 29th for this code

## I cannot get this section to run and I am unsure what I am missing, I attempted to have ChatGPT check this by plugging in my 
# code from september 22 and septmber 29th class and explain the steps so I could change them and apply them for this scenario
# but I was not able to get this code section to run at all. I also plugged in the error message and it was not much help. 
enriched_posts_df = posts_df.merge(
    all_profiles,
    left_on="author_did",
    right_on="did",
)

final_df.head(10)

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

## Section 5 — Conclusion

Describe any patterns you observe in the collected data and how they relate to your hypothesis.
Describe challenges you faced.

Unfortunately, I ran into a lot of errors in this assignment. This was very challenging for me, and I feel like I end up understanding what we do in class, and then when I need to do it on my own I am able to get somewhere using the old codes I have from class, but then when I get an error I do not know howto resolve it. Steps one and two went fine for me , and I felt confident, and then step three was going okay until my last big section of code only came back with errors. This is where I started getting confused and I feel as though neither my notes or old jupyter notebooks or AI was able to help me clarify this. It could be because I spent a few hours trying to change and fix it so much that I almost got confused with what my original code even was, but this is where this assignment started to go bad for me. Section four I was not really able to complete due to the error messages I was recieving, and I was also not able to do a scatter plot even though I was planning to try one in order to help me visualize the data and see a pattern between my two endpoints and test my hypothesis. Unfortunately I was not able to get the visualizations I wanted for this assignment and get past the "Error fetching profile for did:plc:rjscsr7qexxigyhnjctym4bk: 401 Client Error: Unauthorized for url: https://bsky.social/xrpc/app.bsky.actor.getProfile?actor=did%3Aplc%3Arjscsr7qexxigyhnjctym4bk" error, which led me to not be able to run my code and also not be able to let me get the data points needed. I was also confused since when I was running my code earlier last week in class I was getting some results back with the post count saying none, so I am curious if that was something I would have run into here. I believe that my hypothesis would still be true that the more followers someone has they also have more posts, but unfortunately I do not have a way to prove this. 

In [95]:
#  code from sept 15th for reference (
   # tips['day']
  #  .value_counts()
   # .reset_index(name='count')
   # .rename(columns={'index':'day'})
   # .plot(kind='bar',x='day',y='count', title='Rows per day')
#); plt.show

import matplotlib.pyplot as plt

# Base scatterplot template
plt.scatter(
    enriched_posts_df['postsCount'], 
    enriched_posts_df['followersCount']  
)

plt.xlabel("Number of Posts")      
plt.ylabel("Number of Followers")      
plt.title("Followers vs Likes")  

plt.show()

# this is not running due to the error I got in the above section of code :( . 

NameError: name 'enriched_posts_df' is not defined